In [1]:
!pip install -q torchtext==0.16.0

In [2]:
!pip install langid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941172 sha256=f1c87d63fc9860ae8bbac7f0558818d1f63e96f8d343ee9faeb427654bc659a9
  Stored in directory: /root/.cache/pip/wheels/23/c8/c6/eed80894918490a175677414d40bd7c851413bbe03d4856c3c
Successfully built langid


## 1. Tải về bộ dữ liệu NTC-SCV

In [3]:
!git clone https://github.com/congnghia0609/ntc-scv.git

Cloning into 'ntc-scv'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 39 (delta 0), reused 4 (delta 0), pack-reused 35
Receiving objects: 100% (39/39), 186.94 MiB | 17.05 MiB/s, done.
Resolving deltas: 100% (9/9), done.
Updating files: 100% (11/11), done.


In [4]:
!unzip /content/ntc-scv/data/data_test.zip -d ./data

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: ./data/data_test/test/pos/10000.txt  
  inflating: ./data/data_test/test/pos/10018.txt  
  inflating: ./data/data_test/test/pos/10020.txt  
  inflating: ./data/data_test/test/pos/10021.txt  
  inflating: ./data/data_test/test/pos/10031.txt  
  inflating: ./data/data_test/test/pos/10042.txt  
  inflating: ./data/data_test/test/pos/10047.txt  
  inflating: ./data/data_test/test/pos/10070.txt  
  inflating: ./data/data_test/test/pos/10075.txt  
  inflating: ./data/data_test/test/pos/10080.txt  
  inflating: ./data/data_test/test/pos/10083.txt  
  inflating: ./data/data_test/test/pos/10091.txt  
  inflating: ./data/data_test/test/pos/10099.txt  
  inflating: ./data/data_test/test/pos/1010.txt  
  inflating: ./data/data_test/test/pos/10103.txt  
 extracting: ./data/data_test/test/pos/10109.txt  
  inflating: ./data/data_test/test/pos/10110.txt  
  inflating: ./data/data_test/test/pos/10134.txt  
  inflating: ./data/data_t

In [5]:
!unzip /content/ntc-scv/data/data_train.zip -d ./data

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: ./data/data_train/train/pos/31228.txt  
  inflating: ./data/data_train/train/pos/3123.txt  
  inflating: ./data/data_train/train/pos/31232.txt  
  inflating: ./data/data_train/train/pos/31233.txt  
  inflating: ./data/data_train/train/pos/31234.txt  
  inflating: ./data/data_train/train/pos/31236.txt  
  inflating: ./data/data_train/train/pos/31238.txt  
  inflating: ./data/data_train/train/pos/31240.txt  
  inflating: ./data/data_train/train/pos/31242.txt  
  inflating: ./data/data_train/train/pos/31247.txt  
  inflating: ./data/data_train/train/pos/31250.txt  
  inflating: ./data/data_train/train/pos/31251.txt  
  inflating: ./data/data_train/train/pos/31252.txt  
  inflating: ./data/data_train/train/pos/31257.txt  
  inflating: ./data/data_train/train/pos/31260.txt  
  inflating: ./data/data_train/train/pos/31261.txt  
  inflating: ./data/data_train/train/pos/31262.txt  
  inflating: ./data/data_train/train/pos/31

In [6]:
import os
import pandas as pd

In [7]:
def load_data_from_path(folder_path):
  examples = []
  for label in os.listdir(folder_path):
    full_path = os.path.join(folder_path, label)
    for file_name in os.listdir(full_path):
      file_path = os.path.join(full_path, file_name)
      with open(file_path, "r", encoding = 'utf-8') as f:
        lines = f.readlines()
      sentence = " ".join(lines)
      if label == 'neg':
        label = 0
      if label == 'pos':
        label = 1
      data = {
          'sentence': sentence,
          'label': label
      }
      examples.append(data)
  return pd.DataFrame(examples)

In [8]:
folder_paths = {
    'train': './data/data_train/train',
    'valid': './data/data_train/test',
    'test': './data/data_test/test'
}

train_df = load_data_from_path(folder_paths['train'])
valid_df = load_data_from_path(folder_paths['valid'])
test_df = load_data_from_path(folder_paths['test'])

In [9]:
train_df.shape

(30000, 2)

In [11]:
valid_df.shape

(10000, 2)

In [12]:
test_df.shape

(10000, 2)

## 2. Tiền xử lí dữ liệu:

In [ ]:
# Xoá bỏ những bình luận không phải tiếng việt:
from langid.langid import LanguageIdentifier, model

def identify_vn(df):
  identifier = LanguageIdentifier.from_modelstring(model, norm_probs = True)
  not_vi_idx = set()
  THRESHOLD = 0.9
  for idx, row in df.iterrows():
    score = identifier.classify(row['sentence'])
    if score[0] != "vi" or (score[0] == 'vi' and score[1] <= THRESHOLD):
      not_vi_idx.add(idx)
    vi_df = df[~df.index.isin(not_vi_idx)]
    not_vi_df = df[df.index.isin(not_vi_idx)]
    return vi_df, not_vi_df

In [ ]:
train_df_vi, train_df_other = identify_vn(train_df)

In [ ]:
# Làm sạch dữ liệu:
import re
import string

def preprocess_text(text):
    url_pattern = re.compile(r'https?://\s+\wwww\.\s+')
    text = url_pattern.sub(" ", text)

    html_pattern = re.compile(r'<[^<>]+>')
    text = html_pattern.sub(" ", text)

    replace_chars = list(string.punctuation + string.digits)
    for char in replace_chars:
        text = text.replace(char, " ")

    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(" ", text)

    text = " ".join(text.split())

    return text.lower()

In [ ]:
train_df_vi['preprocess_sentence'] = [preprocess_text(row['sentence']) for index, row in train_df_vi.iterrows()]
valid_df['preprocess_sentence'] = [preprocess_text(row['sentence']) for index, row in valid_df.iterrows()]
test_df['preprocess_sentence'] = [preprocess_text(row['sentence']) for index, row in test_df.iterrows()]


## 3. Biểu diễn văn bản thành vector:

In [ ]:
# word-based tokenizer:
from torchtext.data.utils import get_tokenizer
tokenizer = get_tokenizer('basic_english')

# create iter dataset:
def yield_tokens(sentences, tokenizer):
  for sentence in sentences:
    yield tokenizer(sentence)

In [ ]:
# build vocabulary:
from torchtext.vocab import build_vocab_from_iterator

vocab_size = 10000
vocabulary = build_vocab_from_iterator(
    yield_tokens(train_df_vi['preprocess_sentence'], tokenizer),
    max_tokens = vocab_size,
    specials = ["<unk>"]
)
vocabulary.set_default_index(vocabulary['<unk>'])

In [ ]:
# convert iter into torchtext dataset
from torchtext.data.functional import to_map_style_dataset

def prepare_dataset(df):
  for index, row in df.iterrows():
    sentence = row['preprocess_sentence']
    encoded_sentence = vocabulary(tokenizer(sentence))
    label = row['label']
    yield encoded_sentence, label

In [ ]:
train_dataset = prepare_dataset(train_df_vi)
train_dataset = to_map_style_dataset(train_dataset)

valid_dataset = prepare_dataset(valid_df)
valid_dataset = to_map_style_dataset(valid_dataset)

In [ ]:
# Đoạn này để làm một cái gì đó
import torch

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

def collate_batch(batch):
  encoded_sentences, labels, offsets = [], [], [0]
  for encoded_sentence, label in batch:
    labels.append(label)
    encoded_sentence = torch.tensor(encoded_sentence, dtype = torch.int64)
    encoded_sentences.append(encoded_sentence)
    offsets.append(encoded_sentence.size(0))

  labels = torch.tensor(labels, dtype = torch.int64)
  offsets = torch.tensor(offsets[:-1]).cumsum(dim = 0)
  encoded_sentences = torch.cat(encoded_sentences)
  return encoded_sentences.to(device), offsets.to(device), labels.to(device)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 128

train_dataloader = DataLoader(
    train_dataset,
    batch_size = batch_size,
    shuffle = True,
    collate_fn = collate_batch
)

valid_dataloader = DataLoader(
    valid_dataset,
    batch_size = batch_size,
    shuffle = True,
    collate_fn = collate_batch
)

## 4. Xây dựng mô hình phân loại:

In [ ]:
from torch import nn

class TextClassificationModel(nn.Module):
  def __init__(self, vocab_size, embed_dim, num_class):
    super(TextClassificationModel, self).__init__()
    self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse = False)
    self.fc = nn.Linear(embed_dim, num_class)
    self.init_weights()

  def init_weights(self):
    initrange = 0.5
    self.embedding.weight.data.uniform_(-initrange, initrange)
    self.fc.weight.data.uniform_(-initrange, initrange)
    self.fc.bias.data.zero_()

  def forward(self, inputs, offsets):
    embedded = self.embedding(inputs, offsets)
    return self.fc(embedded)

## 5. Huấn luyện mô hình:

In [ ]:
import time

def train(model, optimizer, criterion, train_dataloader, epoch=0, log_interval=25):
    model.train()
    total_acc, total_count = 0, 0
    losses = []
    start_time = time.time()

    for idx, (inputs, offsets, labels) in enumerate(train_dataloader):
        optimizer.zero_grad()
        predictions = model(inputs, offsets)

        # tính toán loss
        loss = criterion(predictions, labels)
        losses.append(loss.item())

        # backward
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()

        total_acc += (predictions.argmax(1) == labels).sum().item()
        total_count += labels.size(0)

        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| độ chính xác {:8.3f}".format(
                    epoch, idx, len(train_dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()

    epoch_acc = total_acc / total_count if total_count != 0 else 0
    epoch_loss = sum(losses) / len(losses) if len(losses) != 0 else 0
    return epoch_acc, epoch_loss


In [ ]:
# evaluation:
def evaluate(model, criterion, valid_dataloader):
  model.eval()
  total_acc, total_count = 0, 0
  losses = []

  with torch.no_grad():
    for idx, (inputs, offsets, labels) in enumerate(valid_dataloader):
      predictions = model(inputs, offsets)
      loss = criterion(predictions, labels)
      losses.append(loss)
      total_acc += (predictions.argmax(1) == labels).sum().item()
      total_count += labels.size(0)

  epoch_acc = total_acc / total_count
  epoch_loss = sum(losses) /len(losses)
  return epoch_acc, epoch_loss

In [ ]:
# start
num_class = len(set(train_df_vi['label']))
vocab_size = len(vocabulary)
embed_dim = 100
model = TextClassificationModel(vocab_size, embed_dim, num_class).to(device)

learning_rate = 5
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

num_epochs = 5
for epoch in range(1,num_epochs+1):
  epoch_start_time = time.time()
  train_acc, train_loss = train(model, optimizer, criterion, train_dataloader, epoch)
  eval_acc, eval_loss = evaluate(model, criterion, valid_dataloader)
  print("-"*59)
  print(
    "| Kết thúc epoch {:3d} | Thời gian: {:5.2f}s | Độ chính xác Huấn luyện {:8.3f} | Loss Huấn luyện {:8.3f} |"
    .format(epoch, time.time() - epoch_start_time, train_acc, train_loss)
  )

  print(
    "| Độ chính xác Kiểm tra {:8.3f} | Loss Kiểm tra {:8.3f} |"
    .format(eval_acc, eval_loss)
  )

  print("-" * 59)



| epoch   1 |    25/  235 batches | độ chính xác    0.600
| epoch   1 |    50/  235 batches | độ chính xác    0.744
| epoch   1 |    75/  235 batches | độ chính xác    0.779
| epoch   1 |   100/  235 batches | độ chính xác    0.820
| epoch   1 |   125/  235 batches | độ chính xác    0.815
| epoch   1 |   150/  235 batches | độ chính xác    0.835
| epoch   1 |   175/  235 batches | độ chính xác    0.831
| epoch   1 |   200/  235 batches | độ chính xác    0.838
| epoch   1 |   225/  235 batches | độ chính xác    0.850
-----------------------------------------------------------
| Kết thúc epoch   1 | Thời gian:  1.26s | Độ chính xác Huấn luyện    0.849 | Loss Huấn luyện    0.461 |
| Độ chính xác Kiểm tra    0.850 | Loss Kiểm tra    0.380 |
-----------------------------------------------------------
| epoch   2 |    25/  235 batches | độ chính xác    0.852
| epoch   2 |    50/  235 batches | độ chính xác    0.849
| epoch   2 |    75/  235 batches | độ chính xác    0.858
| epoch   2 |   100

## 6. Dự đoán và đánh giá mô hình:

In [ ]:
def predict(text):
  with torch.no_grad():
    encoded = torch.tensor(vocabulary(tokenizer(text)))
    output = model(encoded, torch.tensor([0]))
    return output.argmax(1).item()

In [ ]:
predictions, labels = [], []
for index, row in test_df.iterrows():
  sentence = row['preprocess_sentence']
  label = row['label']
  prediction = predict(sentence)
  predictions.append(prediction)
  labels.append(label)

sum(torch.tensor(predictions) == torch.tensor(labels)) /len(labels)

tensor(0.8723)